# リスコフの置換原則（**L**iskov Substitution Principle：LSP）

**派生型はその基本型と置換可能でなければならない**

*   基本型：親クラス、スーパークラス（のオブジェクト）
*   派生型：子クラス、サブクラス（のオブジェクト）

In [ ]:
# 置換可能なケース
class Bird:
    def fly(self) -> str:
        return '飛びます'

class Penguin(Bird):
    def fly(self) -> str:
        return '飛べません'

def bird_fly(bird: Bird):
    print(bird.fly())

bird = Bird()
penguin = Penguin()

#　基本型と派生型を同様に扱えている
bird_fly(bird)
bird_fly(penguin)

飛びます
飛べません


In [ ]:
#　置換不可能なケース
class Bird:
    def fly(self) -> str:
        return '飛びます'

class Penguin(Bird):
    def fly(self) -> None:
        raise Exception('飛べません')

def bird_fly(bird: Bird):
    print(bird.fly())

bird = Bird()
penguin = Penguin()

bird_fly(bird)
bird_fly(penguin) # 例外が発生する

飛びます


Exception: ignored

置換可能なケースでは、基本型と派生型を同様に扱えているが、


置換不可能なケースでは、派生型を基本型と同様に扱おうとすると例外が発生する

⇒　派生型を基本型と置換できるように定義しよう！というのがLSP

## LSPのメリット

LSPを満たして、派生型が基本型と置換可能になると、

**サブタイプを切り替えられる**ようになる

⇒　OCPを実現できる継承が可能になる

In [ ]:
#　Strategyパターンのコード（再掲）
from abc import ABC, abstractmethod

class AbstractNotification(ABC):
    @abstractmethod
    def send(self, user_id: int) -> None:
        pass

class EmailNotification(AbstractNotification):
    def send(self, user_id: int) -> None:
        # メールで通知を送信
        print('メール')

class SMSNotification(AbstractNotification):
    def send(self, user_id: int) -> None:
        # SMSで通知を送信
        print('SMS')

class PushNotification(AbstractNotification):
    def send(self, user_id: int) -> None:
        # プッシュ通知を送信
        print('プッシュ通知')

# 派生型を切り替えることができる
def notify(user_id: int, notification: AbstractNotification): #　AbstractNotificationのサブクラスだけ渡せる
    notification.send(user_id) # すべての派生型において、sendメソッドを同様に使える

継承はもともと、コードの再利用による、保守性の向上を目的として誕生したが、

実際には、保守性を低下させるような使われ方が非常に多かった（[神クラス](https://en.wikipedia.org/wiki/God_object)など）

⇒　最近のプログラミング言語では、クラスの継承を禁止している言語もある（Go / Rustなど）

⇒　LSPを守るように継承を利用すれば、OCPを守れるようになり、変更に強くなる

## 置換できないのは、どのような場合か？

次のいずれかが起こると、基本型を派生型で置き換えることができなくなる

1.   派生型で、基本型よりも**事前条件**を強める
2.  派生型で、基本型よりも**事後条件**を弱める
3.  派生型で、基本型の**不変条件**に違反する
4.  派生型で、基本型にない例外が発生する（1と同じ）

事前条件・事後条件・不変条件は、「**契約による設計**」の構成要素

### 契約による設計（Design by Contract：DbC）




**呼び出される機能と、呼び出し側が、契約を結ぶ（と考える）ことで、**

**ソフトウェアの正確性と頑健性を高める技法**

契約による設計は、次の3つの要素からなる

*   事前条件（precondition）：関数（メソッド）の開始時に保証されるべき条件
*   事後条件（postcondition）：関数（メソッド）の正常終了時に満たされるべき条件
*   不変条件（invariant）：関数（メソッド）の開始時と正常終了時に共通して保証されるべき状態についての条件。クラス不変表明。

呼び出される機能と、呼び出し側は、次の契約を結ぶ（と考える）



> 呼び出し側によって、呼び出される機能の事前条件がすべて満足された場合、
>
> 当該機能は処理完了時点ですべての事後条件と不変表明を満足させるものとする。


上記は、[達人プログラマー](https://hiramatsuu.com/archives/1433)より引用


上記を言い換えると、
>関数やメソッドが呼び出されるときには、必ず事前条件を満たした状態で関数やメソッドを呼び出す。
>
>かつ、事前条件が満たされた状態で関数やメソッドが呼び出された場合、必ず事後条件と不変条件を満たすように関数やメソッドを定義する。

銀行口座を表現するクラスについて考えてみよう

In [ ]:
# 契約による設計を意識していないコード
class BankAccount:
    def __init__(self) -> None:
        self._balance = 0 # 口座の初期残高は0

    def deposit(self, amount: int) -> None:
        self._balance += amount

    def withdraw(self, amount: int) -> None:
        self._balance -= amount

上記のコードでは、次のような問題点がある

*   入出金額を負の値にできてしまう
*   残高が0より小さくなる可能性がある
*   残高をクラスの外部から変更できる

これらの問題は、銀行口座が持つ以下のようなルールが、

コードに反映されていないために起こる

*   入出金額は0以上の整数値（事前条件）
*   残高以上の額を出金することはできない（事前条件）
*   入金・出金額と口座残高が整合性を保つ必要がある（事後条件）
*   残高は常に0以上の整数値（不変条件）

In [ ]:
# 事前条件を守れるように変更したコード（ガード節を追加）
class BankAccount:
    def __init__(self) -> None:
        self._balance = 0

    def deposit(self, amount: int) -> None:
        if amount < 0:
            raise ValueError('事前条件: 入金額は0以上')
        self._balance += amount

    def withdraw(self, amount: int) -> None:
        if amount < 0:
            raise ValueError('事前条件: 出金額は0以上')
        if amount > self._balance:
            raise ValueError('事前条件: 引き出し額は現在の残高以下')
        self._balance -= amount

**契約に違反した場合は、例外を発生させる**ことで、

契約に違反したまま処理が続くことを防げる

In [ ]:
# 事後条件を守れるように変更したコード
class BankAccount:
    def __init__(self) -> None:
        self._balance = 0

    def deposit(self, amount: int) -> None:
        if amount < 0:
            raise ValueError('事前条件: 入金額は0以上')

        old_balance = self._balance
        new_balance = self._balance + amount
        self._check_postcondition(new_balance, old_balance, amount) # 事後条件をチェック
        self._balance = new_balance

    def withdraw(self, amount: int) -> None:
        if amount < 0:
            raise ValueError('事前条件: 出金額は0以上')
        if amount > self._balance:
            raise ValueError('事前条件: 引き出し額は現在の残高以下')

        old_balance = self._balance
        new_balance = self._balance - amount
        self._check_postcondition(new_balance, old_balance, amount) # 事後条件をチェック
        self._balance = new_balance

    #　事後条件を確認するメソッド
    def _check_postcondition(self, new_balance: int, old_balance: int, amount: int):
        difference = new_balance - old_balance
        if abs(difference) != amount:
            raise ValueError('事後条件: 入金・出金額と口座残高に整合性がある')

実用の場面において、事後条件は、

事前条件や不変条件のように、ガード節として組み込まれるというよりは、

**テストケースとして確認される**のが基本

In [ ]:
# 不変条件を守れるように変更したコード
class BankAccount:
    def __init__(self) -> None:
        self._balance = 0

    # クラス外部から変更されないようにする（もどき） (getter)
    @property
    def balance(self) -> int:
        return self._balance

    def deposit(self, amount: int) -> None:
        if amount < 0:
            raise ValueError('事前条件: 入金額は0以上')

        new_balance = self._balance + amount
        self._check_invariant(new_balance) # 不変条件をチェック
        self._balance = new_balance

    def withdraw(self, amount: int) -> None:
        if amount < 0:
            raise ValueError('事前条件: 出金額は0以上')
        if amount > self._balance:
            raise ValueError('事前条件: 引き出し額は現在の残高以下')

        new_balance = self._balance - amount
        self._check_invariant(new_balance) # 不変条件をチェック
        self._balance = new_balance

    #　不変条件を確認するメソッド
    def _check_invariant(self, new_balance):
        if new_balance < 0:
            raise ValueError('不変条件: 残高は0以上')

契約による設計を意識すると、コードがより正確・頑健になる

## LSPに違反した継承の例

（再掲）次のいずれかが起こると、基本型を派生型で置き換えることができなくなる

1.  派生型(継承先)で、基本型(継承元)よりも**事前条件**を強める
			
			(継承先の条件が継承元の事前条件よりも強くなっている)
2.  派生型(継承先)で、基本型(継承元)よりも**事後条件**を弱める
			
			(継承先の条件が継承元の事後条件よりも弱くなっている)
3.  派生型(継承先)で、基本型(継承元)の**不変条件**に違反する
			
			(継承先の条件が継承元の不変条件に違反している)
4.  派生型(継承先)で、基本型(継承元)にない例外が発生する（1と同じ）
			
			(継承先の条件が継承元の例外に違反している)

### 事前条件を派生型で強めている

In [ ]:
# LSPに違反した継承パターンの1と4に当てはまる例

# 事前条件を派生型で強めているパターン
class SuperHoge:
    def hoge(self, n: int) -> int:
        return n

# SuperHogeに比べてSubHogeの条件が強くなっている
class SubHoge(SuperHoge):
    def hoge(self, n: int) -> int:
        if n < 0:
            raise ValueError('0以上にしてください')
        return n

「派生型で、基本型にない例外が発生する」ということは、

派生型で基本型の事前条件を強めていることになるので、

これも基本型と派生型を置換不可能にする

In [ ]:
# LSPに違反した継承パターンの4のみに当てはまる例

# 事前条件を派生型で強めているパターン（例外の発生なし）
class SuperHoge:
    def hoge(self, n: int) -> int:
        return n

class SubHoge(SuperHoge):
    def hoge(self, n: int) -> int:
        #　nは0以上を期待しているが、ガード節がない
        return n

逆に以下のコードは問題ない

In [ ]:
# 事前条件を派生型で弱めているパターン (OKなパターン)
class SuperHoge:
    def hoge(self, n: int) -> int:
        if n < 0:
            raise ValueError('0以上にしてください')
        return n

class SubHoge(SuperHoge):
    def hoge(self, n: int) -> int:
        return n

def hoge0(hoge_obj: SuperHoge):
    print(hoge_obj.hoge(0))

基本型が正常動作する呼び出し方を、

派生型でも同様に行って問題ないなら、置換可能

### 事後条件を派生型で弱めている

In [ ]:
#　事後条件を派生型で弱めているパターン
class SuperHoge:
    def hoge(self, n: int) -> int:
        return n

class SubHoge(SuperHoge):
    # 派生型で事後条件を弱めている (戻り値の範囲が広くなっている(戻り値の型が複数になっている))
    # 戻り値がn>0の場合は、tuple型でintとstrを返すようになっており、
    # n<=0の場合は、int型でnを返すようになっている
    def hoge(self, n: int) -> int | tuple[int, str]: # 戻り値の範囲が広くなっている
        if n > 0:
            return n , '正の数です' # tuple型で返す (tuple[int, str])
        return n

def hoge_client(hoge_obj: SuperHoge):
    return hoge_obj.hoge(2) * 10

戻り値の型にいくつかのパターンがある場合は、次のように書ける



```
def メソッド名(self) -> 型1 | 型2:
```



詳しくは、[公式ドキュメント](https://docs.python.org/ja/3/library/stdtypes.html#union-type)を参照

逆に以下のコードは問題ない

In [ ]:
#　事後条件を派生型で強めているパターン (OKなパターン)
class SuperHoge:
    def hoge(self, n: int) -> int:
        return n

class SubHoge(SuperHoge):
    # 派生型で事後条件を強めている
    # 戻り値は継承元と同じint型で返す
    # しかし、派生型では負の数はすべて0にして返すようになっている (戻り値の範囲が狭くなっている = 事後条件を強めている)
    def hoge(self, n: int) -> int:
        if n < 0:
            return 0 # 負の数はすべて0にして返す、戻り値の範囲が狭くなっている
        return n

### 不変条件に派生型で違反している

In [ ]:
# 不変条件に派生型で違反しているパターン
# 基本型(継承元: SuperHoge)で守れられている条件が、派生型(継承先: SubHoge)で守れなくなっている

class SuperHoge:
    def __init__(self, n: int) -> None:
        if n < 1:
            raise ValueError('1以上にしてください') # 不変条件、nは１以上
        self._n = n

    def hoge(self, n: int) -> int:
        #　事前条件
        if n < 1:
            raise ValueError('1以上にしてください')
        self._n = self._n * n
        return self._n

class SubHoge(SuperHoge):
    def hoge(self, n: int) -> int:
        #　事前条件
        if n < 1:
            raise ValueError('1以上にしてください')
        self._n = self._n // n  # nが１よりも小さくなる可能性がある (例: self.n = 2, n = 4, 結果=0)
        return self._n

### ちょっとクイズ

①次のコードは、LSPに違反しているか？

In [ ]:
class Rectangle: # 長方形クラス
    def __init__(self, width: int, height: int) -> None:
        self.width = width
        self.height = height

    def area(self) -> int:
        return self.width * self.height

class Square(Rectangle): # 正方形クラス
    def __init__(self, side: int) -> None:
        super().__init__(side, side)

**答え**

クライアントを見ないとわからない (基本型(継承元: Rectangle)と派生型(継承先:Square)を見ただけでは判断できない

②次のコードは、

adjust_width関数から見た時に、

LSPに違反しているか？

In [ ]:
class Rectangle: # 長方形クラス
    def __init__(self, width: int, height: int) -> None:
        self.width = width
        self.height = height

    def area(self) -> int:
        return self.width * self.height

class Square(Rectangle): # 正方形クラス
    def __init__(self, side: int) -> None:
        super().__init__(side, side)

def adjust_width(rectangle: Rectangle, new_width: int) -> None:
    print(f'初期の面積: {rectangle.area()}')
    rectangle.width = new_width
    print(f'幅を調整後の面積: {rectangle.area()}')

答え：違反している

理由: 長方形では期待通りの振る舞いを行うが、正方形の場合だと期待とは異なる結果になる

In [ ]:
def adjust_width(rectangle: Rectangle, new_width: int) -> None:
    print(f'初期の面積: {rectangle.area()}')
    rectangle.width = new_width
    print(f'幅を調整後の面積: {rectangle.area()}')

rectangle = Rectangle(5, 10)
adjust_width(rectangle, 7)

square = Square(5)
adjust_width(square, 7) # 期待とは異なる結果になる
                        # 期待は49になるが、結果は35になる

初期の面積: 50
幅を調整後の面積: 70
初期の面積: 25
幅を調整後の面積: 35


用語の定義からすると「正方形 is a 長方形（正方形は長方形の1つ）」なのは間違いないが、

用語が「is a」の関係だからといって、適切な継承ができているとは限らない

⇒　**クライアントから見て、「is a」の関係が成立している**必要がある

⇒　「クライアントから見て」LSPを満たすか？が判断基準

次のようなクライアントから見れば、

上記の基本型・派生型もLSPを満たしていると言える

In [ ]:
def print_area(rectangle: Rectangle) -> None:
    print(f'面積: {rectangle.area()}')

rectangle = Rectangle(5, 10)
print_area(rectangle)

square = Square(5)
print_area(square)

面積: 50
面積: 25


## 他の原則との関係性


OCPを実現するための、継承の使い方を示すのがLSP

## LSPのまとめ

*   リスコフの置換原則（LSP）は、「派生型はその基本型と置換可能でなければならない」という原則
*   LSPを守ることで、サブタイプを切り替えられるようになり、OCPを満たす継承ができるようになる
*   派生型が基本型と置換可能かどうかを判断するには、
  
    契約による設計（Design by Contract：DbC）の観点が役立つ
*   事前条件（precondition）：関数（メソッド）の開始時に保証されるべき条件
*   事後条件（postcondition）：関数（メソッド）の正常終了時に満たされるべき条件
*   不変条件（invariant）：関数（メソッド）の開始時と正常終了時に共通して保証されるべき状態についての条件
*   例外の使い所は、契約を満たせなくなったとき
*   派生型で基本型の①事前条件を強める、②事後条件を弱める、③不変条件に違反する、とLSPに違反してしまう
*   LSPは、クライアントから見た「is aの関係」を保証するための原則とも言える


## LSPの演習問題

①LSPとはどのような原則でしょうか

これまでに学んだことを、できる限り多く思い出してみましょう

②基本型を派生型で置換できないのはどのような場合でしょうか

契約による設計の観点から回答してください

③商品を表現するProductクラスについて考えてみましょう

商品は以下のルールを持つとします

*   名前は1文字以上20文字以内
*   価格は1円以上
*   割引率は5~50%

次のコードに契約による設計の考え方を取り入れて、

コードを改善してください

ただし、新たなクラスは定義しないものとします

In [ ]:
class Product:
    def __init__(self, name: str, price: int) -> None:
        self._name = name
        self._price = price

    def discount(self, discount_percent: int) -> None:
        self._price = self._price * (100 - discount_percent) // 100

    def change_name(self, new_name: str) -> None:
        self._name = new_name

④次のコードはLSPに違反しているでしょうか

違反している場合は、その理由を述べた上で、

LSPを満たすようにコードを改善してください

In [ ]:
from abc import ABC, abstractmethod

class CouponStrategy(ABC):
    @abstractmethod
    def apply_discount(self, price: int) -> int:
        pass

class PercentageDiscountCoupon(CouponStrategy):
    def __init__(self, percentage: int):
        self.percentage = percentage

    def apply_discount(self, price: int) -> int:
        return int(price * (1 - self.percentage / 100))

class QuantityDiscountCoupon(CouponStrategy):
    def apply_discount(self, price: int, items_count: int) -> int:
        if items_count >= 10:
            return int(price * 0.9)
        return price

class ShoppingCart:
    def __init__(self, discount_strategy: CouponStrategy):
        self.items = []
        self.discount_strategy = discount_strategy

    def add_item(self, item_name: str, price: int):
        self.items.append((item_name, price))

    def calculate_total(self) -> int:
        total = sum(price for _, price in self.items)
        return self.discount_strategy.apply_discount(total)

⑤ ④の解答のコードの問題点を指摘し、

次のコードに記載されているクラスを活用して改善してください

In [ ]:
class DiscountParameters:
    def __init__(self, items: list):
        self._items = items

    @property
    def total_price(self) -> int:
        return sum(price for _, price in self._items)

    @property
    def item_count(self) -> int:
        return len(self._items)

## リスコフの置換原則（**L**iskov Substitution Principle：LSP）

①LSPとはどのような原則でしょうか

これまでに学んだことを、できる限り多く思い出してみましょう

②基本型を派生型で置換できないのはどのような場合でしょうか

契約による設計の観点から回答してください

次のいずれかが起こると、基本型を派生型で置き換えることができなくなる

1.   派生型で、基本型よりも事前条件を強める
2.  派生型で、基本型よりも事後条件を弱める
3.  派生型で、基本型の不変条件に違反する

③商品を表現するProductクラスについて考えてみましょう

商品は以下のルールを持つとします

*   名前は1文字以上20文字以内
*   価格は1円以上
*   割引率は5~50%

次のコードに契約による設計の考え方を取り入れて、

コードを改善してください

ただし、新たなクラスは定義しないものとします

In [ ]:
class Product:
    def __init__(self, name: str, price: int) -> None:
        self._name = name
        self._price = price

    def discount(self, discount_percent: int) -> None:
        self._price = self._price * (100 - discount_percent) // 100

    def change_name(self, new_name: str) -> None:
        self._name = new_name

In [ ]:
# 解答の一例
class Product:
    def __init__(self, name: str, price: int) -> None:
        if not 1 <= len(name) <= 20: # 不変条件
            raise ValueError('名前は1文字以上20文字以内')
        if price < 1: # 不変条件
            raise ValueError('価格は１円以上')
        self._name = name
        self._price = price

    @property # 読み取り専用（もどき）にして外部から変更不可に（不変条件の維持）
    def name(self) -> str:
        return self._name

    @property # 読み取り専用（もどき）にして外部から変更不可に（不変条件の維持）
    def price(self) -> int:
        return self._price

    def discount(self, discount_percent: int) -> None:
        if not 5 <= discount_percent <= 50: # 事前条件
            raise ValueError('割引率は5~50%')
        self._price = self._price * (100 - discount_percent) // 100

    def change_name(self, new_name: str) -> None:
        if not 1 <= len(new_name) <= 20: #　事前条件・不変条件
            raise ValueError('名前は1文字以上20文字以内')
        self._name = new_name

事後条件は、

事前条件や不変条件のように、ガード節として組み込まれるというよりは、

**テストケースとして確認される**のが基本

④次のコードはLSPに違反しているでしょうか

違反している場合は、その理由を述べた上で、

LSPを満たすようにコードを改善してください

In [ ]:
from abc import ABC, abstractmethod

class CouponStrategy(ABC):
    @abstractmethod
    def apply_discount(self, price: int) -> int:
        pass

class PercentageDiscountCoupon(CouponStrategy):
    def __init__(self, percentage: int):
        self.percentage = percentage

    def apply_discount(self, price: int) -> int:
        return int(price * (1 - self.percentage / 100))

class QuantityDiscountCoupon(CouponStrategy):
    def apply_discount(self, price: int, items_count: int) -> int:
        if items_count >= 10:
            return int(price * 0.9)
        return price

class ShoppingCart:
    def __init__(self, discount_strategy: CouponStrategy):
        self.items = []
        self.discount_strategy = discount_strategy

    def add_item(self, item_name: str, price: int):
        self.items.append((item_name, price))

    def calculate_total(self) -> int:
        total = sum(price for _, price in self.items)
        return self.discount_strategy.apply_discount(total)

QuantityDiscountCouponクラスのapply_discountメソッドのみ、

items_countという引数を渡す必要があり、

これは派生型で事前条件を強めていることになるので、LSPに違反する

In [ ]:
# 使用例
cart = ShoppingCart(PercentageDiscountCoupon(10))
cart.add_item('Tシャツ', 2000)
cart.add_item('靴', 5000)
print(f'{cart.calculate_total()}円')

cart2 = ShoppingCart(QuantityDiscountCoupon())
cart2.add_item('Tシャツ', 2000)
cart2.add_item('靴', 5000)
print(f'{cart2.calculate_total()}円')

6300円


TypeError: ignored

In [ ]:
# 解答の一例
from abc import ABC, abstractmethod

class CouponStrategy(ABC):
    @abstractmethod
    def apply_discount(self, price: int, items_count: int = 0) -> int: # 引数を追加
        pass

class PercentageDiscountCoupon(CouponStrategy):
    def __init__(self, percentage: int):
        self.percentage = percentage

    def apply_discount(self, price: int, items_count: int = 0) -> int:
        return int(price * (1 - self.percentage / 100))

class QuantityDiscountCoupon(CouponStrategy):
  def apply_discount(self, price: int, items_count: int) -> int:
        if items_count >= 10:
            return int(price * 0.9)
        return price

class ShoppingCart:
    def __init__(self, discount_strategy: CouponStrategy):
        self.items = []
        self.discount_strategy = discount_strategy

    def add_item(self, item_name: str, price: int):
        self.items.append((item_name, price))

    def calculate_total(self) -> int:
        total = sum(price for _, price in self.items)
        items_count = len(self.items)
        return self.discount_strategy.apply_discount(total, items_count)

# 使用例
cart = ShoppingCart(PercentageDiscountCoupon(20))
cart.add_item('Tシャツ', 2000)
cart.add_item('靴', 5000)
print(f'{cart.calculate_total()}円')

cart2 = ShoppingCart(QuantityDiscountCoupon())
cart2.add_item('Tシャツ', 2000)
cart2.add_item('靴', 5000)
print(f'{cart2.calculate_total()}円')

5600円
7000円


⑤ ④の解答のコードを、

次のコードに記載されているクラスを活用して改善してください

In [ ]:
class DiscountParameters:
    def __init__(self, items: list):
        self._items = items

    @property
    def total_price(self) -> int:
        return sum(price for _, price in self._items)

    @property
    def item_count(self) -> int:
        return len(self._items)

④で見たコードは、1つの派生型でしか使わない引数を、

すべての派生型で実装するように強制してしまっている

⇒　ある派生型に固有の引数が増えるたびに、

　　すべての派生型の引数の変更が必要になってしまう

この問題は、**Parameter Object（Options Object）**パターンで解決できる

### Parameter Object（Options Object）パターン

複数のパラメータ（引数）を、１つのオブジェクトにまとめることで隠蔽するパターン

In [ ]:
from abc import ABC, abstractmethod

# 引数のクラスを用意
class DiscountParameters:
    def __init__(self, items: list):
        self._items = items

    @property
    def total_price(self) -> int:
        return sum(price for _, price in self._items)

    @property
    def item_count(self) -> int:
        return len(self._items)

class CouponStrategy(ABC):
    @abstractmethod
    def apply_discount(self, context: DiscountParameters) -> int: # 引数をオブジェクトにする
        pass

class PercentageDiscountCoupon(CouponStrategy):
    def __init__(self, percentage: int):
        self.percentage = percentage

    def apply_discount(self, context: DiscountParameters) -> int:
        return int(context.total_price * (1 - self.percentage / 100)) # 必要な属性だけを利用

class QuantityDiscountCoupon(CouponStrategy):
    def apply_discount(self, context: DiscountParameters) -> int:
        if context.item_count >= 10:
            return int(context.total_price * 0.9)
        return context.total_price

class ShoppingCart:
    def __init__(self, discount_strategy: CouponStrategy):
        self.items = []
        self.discount_strategy = discount_strategy

    def add_item(self, item_name: str, price: int):
        self.items.append((item_name, price))

    def calculate_total(self) -> int:
        context = DiscountParameters(self.items) # パラメータオブジェクトを生成
        return self.discount_strategy.apply_discount(context) # パラメータオブジェクトだけ渡す

# 使用例
cart = ShoppingCart(PercentageDiscountCoupon(10))
cart.add_item('Tシャツ', 2000)
cart.add_item('靴', 5000)
print(f'{cart.calculate_total()}円')

cart2 = ShoppingCart(QuantityDiscountCoupon())
cart2.add_item('Tシャツ', 2000)
cart2.add_item('靴', 5000)
print(f'{cart2.calculate_total()}円')

6300円
7000円


ついでに、**Null Objectパターン**についても解説

### Null Objectパターン

上記のコードでは、割引がない場合の処理が不足している

次のようにNoneで対処することもできるが、

なるべくNoneはコードから追い出す方が望ましい




In [ ]:
class ShoppingCart:
    def __init__(self, discount_strategy: CouponStrategy = None):
        self.items = []
        self.discount_strategy = discount_strategy

    # 以下略
    def add_item(self, item_name: str, price: int):
        self.items.append((item_name, price))

    def calculate_total(self) -> int:
        context = DiscountParameters(self.items)
        # クーポンの使用があるかどうかでの条件分岐
        if self.discount_strategy is not None:
            return self.discount_strategy.apply_discount(context)
        return context.total_price

cart = ShoppingCart()
cart.add_item('Tシャツ', 2000)
cart.add_item('靴', 5000)
print(f'{cart.calculate_total()}円')

7000円


Null Objectパターンを使うことで、

Noneチェックの条件分岐をなくして、他のクーポンと同様に扱えるようになる

In [ ]:
# Null Object（他の派生型と同じインターフェースを持ったNoneの表現）
class NoCoupon(CouponStrategy):
    def apply_discount(self, context: DiscountParameters) -> int:
        return context.total_price

class ShoppingCart:
    def __init__(self, discount_strategy: CouponStrategy):
        self.items = []
        self.discount_strategy = discount_strategy

    def add_item(self, item_name: str, price: int):
        self.items.append((item_name, price))

    def calculate_total(self) -> int:
        # クーポンなしの場合も同様に扱えるようになる
        context = DiscountParameters(self.items)
        return self.discount_strategy.apply_discount(context)

cart = ShoppingCart(NoCoupon()) # クーポンなしの場合
cart.add_item('Tシャツ', 2000)
cart.add_item('靴', 5000)
print(f'{cart.calculate_total()}円')

7000円
